In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 7

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Feature Engineering",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.copy()
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "ph2oimm",  # Maybe pH of the immersion liquid | pH (acidity or alkalinity)
        "Initial Setting Time",
        "flow",  # Maybe flow table test
        "residuo 24 micron",  # Maybe the residue left after passing the material through a sieve
        "R_wp",  # Maybe water to powder ratio
        "CS2",  # 2-day Compressive Strength
        "Soundness",
        # Removing One-Hot encoding variables
        "cem_type_CEM B",
        "cem_type_CEM C",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

## Feature Engineering

In [8]:
# Feature Engineering over Chemical Features
ch_features = ["CaO", "Ca(OH)2", "Periclasio (MgO)", "K2SO4", "Cl-", "SO3"]
df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

# Feature Engineering over Mineralogical Features
mi_features_set1 = [
    "Alite_M1 C3S M1",
    "Alite_M3 C3S M3",
    "Anidrite",
    "Aphthitalite – (K,Na)3(SO4)2",
    "Belite_beta",
    "C3A cub",
    "C3A tot",
    "C3A_ortho",
    "C3S_CS (taglia dei cristalliti C3S)",
    "C4AF",
    "CO2_XRD",
    "Calce libera",
    "Calcite – CaCO3",
    "Emiidrato",
    "Gesso",
    "Langbeinite – MgK2(SO4)2",
    "Quartz",
    "SO3_XRD",
]

df_copy["std_mi_set1_feats"] = df_copy[mi_features_set1].std(ddof=0, axis=1)

<IPython.core.display.Javascript object>

In [9]:
df_copy.describe().T

,count,mean,std,min,25%,50%,75%,max
SO3,957.0,2.969927,0.353675,1.770000,2.700000,2.960000,3.230000,3.910000
Cl-,958.0,0.049674,0.023952,0.011000,0.022000,0.054000,0.070000,0.099000
CS28,958.0,49.042276,6.617852,34.400000,42.500000,50.800000,54.100000,65.400000
Alite_M3 C3S M3,878.0,28.713200,6.014556,14.150000,24.235000,28.435000,32.800000,45.190000
Alite_M1 C3S M1,878.0,29.622392,5.176509,17.630000,25.832500,29.470000,33.065000,45.580000
Alite_Sum C3S tot,878.0,58.334191,7.494005,41.090000,50.690000,60.890000,64.210000,72.960000
Ratio_M1 (rapporto M1/M3),878.0,50.935197,7.078642,33.720000,45.852500,50.920000,56.110000,68.770000
C3S_CS (taglia dei cristalliti C3S),878.0,237.981002,21.463213,143.760000,225.820000,239.145000,251.500000,306.680000
Belite_beta,878.0,8.183468,2.372578,1.430000,6.510000,8.110000,9.747500,17.410000
C3A cub,878.0,2.014772,0.411328,0.730000,1.760000,2.020000,2.270000,4.540000


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [10]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [11]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.934 (0.036)
MAE: -2.268 (0.029)
MAPE: -0.047 (0.001)
R2: 0.803 (0.005)


******
[TEST]
******
RMSE: -3.044 (0.145)
MAE: -2.371 (0.099)
MAPE: -0.049 (0.002)
R2: 0.787 (0.021)




<IPython.core.display.Javascript object>

In [12]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   

  Cross Validation RMSE Test           MAE Test           MAPE Test            \
                        mean       std     mean       std      mean       std   
0   Repeated KFold  3.043564  0.145455   2.3711  0.099183  0.049367  0.002364   

    R2 Test            
       mean       std  
0  0.786538  0.021077

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [13]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.385 (0.262)
MAE: -1.806 (0.195)
MAPE: -0.038 (0.004)
R2: 0.865 (0.031)


******
[TEST]
******
RMSE: -4.017 (0.779)
MAE: -3.008 (0.852)
MAPE: -0.062 (0.015)
R2: 0.564 (0.165)




<IPython.core.display.Javascript object>

In [14]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  4.016826  0.778704  3.007622  0.852423   
1              Repeated KFold  3.043564  0.145455  2.371100  0.099183   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.061952  0.014722  0.564116  0.165366  
1  0.049367  0.002364  0.786538  0.021077

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [15]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.911 (0.073)
MAE: -2.232 (0.045)
MAPE: -0.047 (0.001)
R2: 0.802 (0.012)


******
[TEST]
******
RMSE: -3.487 (0.419)
MAE: -2.727 (0.339)
MAPE: -0.056 (0.007)
R2: 0.703 (0.081)




<IPython.core.display.Javascript object>

In [16]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  4.016826  0.778704  3.007622  0.852423   
1              Repeated KFold  3.043564  0.145455  2.371100  0.099183   
2           Time Series Split  3.487285  0.419428  2.726603  0.338963   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.061952  0.014722  0.564116  0.165366  
1  0.049367  0.002364  0.786538  0.021077  
2  0.056104  0.007073  0.703131  0.080805

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [17]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.852 (0.000)
MAE: 2.199 (0.000)
MAPE: 0.046 (0.000)
R2: 0.818 (0.000)


******
[TEST]
******
RMSE: 3.648 (0.000)
MAE: 2.992 (0.000)
MAPE: 0.063 (0.000)
R2: 0.660 (0.000)




<IPython.core.display.Javascript object>

In [18]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  4.016826  0.778704  3.007622  0.852423   
1           Out of time Split  3.647549  0.000000  2.991686  0.000000   
2              Repeated KFold  3.043564  0.145455  2.371100  0.099183   
3           Time Series Split  3.487285  0.419428  2.726603  0.338963   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.061952  0.014722  0.564116  0.165366  
1  0.062655  0.000000  0.659905  0.000000  
2  0.049367  0.002364  0.786538  0.021077  
3  0.056104  0.007073  0.703131  0.080805

<IPython.core.display.Javascript object>

In [19]:
pipeline.named_steps["estimator"].coef_

array([  1.37690533,  -1.62423022, -13.26126553,  -5.55075647,
        37.53561232,  -3.18597861,   3.94918381,   7.10152778,
        -2.87391015,  -5.02976274,   8.39032576,   5.40203924,
         3.89947866,   3.46389286,  -1.20286352,   0.22997342,
         1.40809861,   0.79215458,   0.72537928,   0.42185466,
         2.3424487 ,   1.57288998,   2.48990368,  25.38109367,
         0.25856264,  -1.40043915,  -1.37832403,  -3.96364036])

<IPython.core.display.Javascript object>

In [20]:
pipeline.named_steps["estimator"].intercept_

49.22584856396869

<IPython.core.display.Javascript object>

In [21]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [22]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Alite_M3 C3S M3', 'Alite_M1 C3S M1', 'Alite_Sum C3S tot',
       'Ratio_M1 (rapporto M1/M3)', 'C3S_CS (taglia dei cristalliti C3S)',
       'Belite_beta', 'C3A cub', 'C3A_ortho', 'C3A tot', 'C4AF', 'CaO',
       'Ca(OH)2', 'Calce libera', 'Periclasio (MgO)', 'Quartz', 'K2SO4',
       'Langbeinite – MgK2(SO4)2', 'Aphthitalite – (K,Na)3(SO4)2', 'Gesso',
       'Emiidrato', 'Anidrite', 'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD',
       'std_ch_feats', 'std_mi_set1_feats'],
      dtype='object')

<IPython.core.display.Javascript object>

In [23]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(28,)

<IPython.core.display.Javascript object>

In [24]:
x.columns.shape

(28,)

<IPython.core.display.Javascript object>

In [25]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [26]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Alite_Sum C3S tot,37.535612
Calcite – CaCO3,25.381094
C3A tot,8.390326
Belite_beta,7.101528
C4AF,5.402039
C3S_CS (taglia dei cristalliti C3S),3.949184
CaO,3.899479
Ca(OH)2,3.463893
Anidrite,2.489904
Gesso,2.342449


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  4.016826  0.778704   
1                                {"Test Size": 0.2}  3.647549  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  3.043564  0.145455   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.487285  0.419428   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  3.007622  0.852423  0.061952  0.014722  0.564116  0.165366  
1  2.991686  0.000000  0.062655  0.000000  0.659905  0.000000  
2  2.371100  0.099183  0.049367  0.002364  0.786538  0.021077  
3  2.726603  0.338963  0.056104  0.007073  0.703131  0.080805

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [28]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [29]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [30]:
df_results_to_save

Category Company Data Shape Timesteps  \
                                              
0  Local Model   partner_iv  (958, 28)       NaN   
1  Local Model   partner_iv  (958, 28)       NaN   
2  Local Model   partner_iv  (958, 28)       NaN   
3  Local Model   partner_iv  (958, 28)       NaN   

                                         Features              Model  \
                                                                       
0  Chemical + Mineralogical + Feature Engineering  Linear Regression   
1  Chemical + Mineralogical + Feature Engineering  Linear Regression   
2  Chemical + Mineralogical + Feature Engineering  Linear Regression   
3  Chemical + Mineralogical + Feature Engineering  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   2.384572  0.262214  ...   
1                                {"Test Size": 0.2}   2.851751  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.934112  0.035865  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.911461  0.072893  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.865331  0.030943  4.016826  0.778704  3.007622  0.852423  0.061952   
1  0.818239  0.000000  3.647549  0.000000  2.991686  0.000000  0.062655   
2  0.803120  0.005020  3.043564  0.145455  2.371100  0.099183  0.049367   
3  0.802109  0.012165  3.487285  0.419428  2.726603  0.338963  0.056104   

              R2 Test            
        std      mean       std  
0  0.014722  0.564116  0.165366  
1  0.000000  0.659905  0.000000  
2  0.002364  0.786538  0.021077  
3  0.007073  0.703131  0.080805  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>